# Loose Entity Extraction

## Setup

In [1]:
%pip install python-dotenv pypdf

Note: you may need to restart the kernel to use updated packages.


In [2]:
from dotenv import load_dotenv
import os
load_dotenv('exp.env', override=True)

# Neo4j
NEO4J_URI = os.getenv('NEO4J_URI')
NEO4J_USERNAME = os.getenv('NEO4J_USERNAME')
NEO4J_PASSWORD = os.getenv('NEO4J_PASSWORD')

#OPENAI
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

## Experimentation

In [3]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("2020-Ford-F250-F350-F450-F550-F600-Owners-Manual-version-1._om_EN_10_2019.pdf")
pages = loader.load_and_split()

In [4]:
len(pages)

648

In [5]:
pages[112]

Document(page_content='Powertrain Malfunction/Reduced\nPower/Electronic Throttle Control\nIlluminates when a powertrain\nor a 4WD fault has been\ndetected. Contact your\nauthorized dealer as soon as possible.\nService Engine Soon\nIf it illuminates when the engine\nis running this indicates a\nmalfunction.  The On Board\nDiagnostics system has detected a\nmalfunction of the vehicle emission control\nsystem.\nIf it flashes, engine misfire may be\noccurring.  Increased exhaust gas\ntemperatures could damage the catalytic\nconverter or other vehicle components.\nDrive in a moderate fashion (avoid heavy\nacceleration and deceleration) and have\nyour vehicle immediately serviced.\nIt illuminates when you switch the ignition\non prior to engine start to check the bulb\nand to indicate whether your vehicle is\nready for Inspection and Maintenance\n(I/M) testing.\nNormally, it illuminates until the engine is\ncranked and automatically turns off if no\nmalfunctions are present. However, if afte

In [7]:
from langchain_core.prompts import ChatPromptTemplate

extract_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are an expert car mechanic that needs to extract relevant facts from the below "
            "text from the Ford Super Duty owners manual. "
            "The extracted facts will be loaded into a knowledge base for helping non-expert owners "
            "answer critical questions about their vehicle and provide them expert guidance. "
            "The facts should be extracted in the form of a a knowledge triple: subject, predicate object."
            "each subject, predicate, and object should only be 1 to a few words. Avoid sentences and conjunctions, "
            "instead you can create more facts. "
            "Avoid object containing a preposition or adverb, instead create additional facts. "
            "Given that this is a source of truth, please ensure the facts come only from the provided text, "
            "Do not create fictitious data or impute missing values."
            "Only include facts that are clear, non-ambiguous, and relevant to the research"
        ),
        ("human", "{text}"),
    ]
)

In [8]:
import datetime
from typing import List
from langchain_core.pydantic_v1 import BaseModel, Field

class Fact(BaseModel):
    """A useful fact"""
    subject: str = Field(description="Subject: Entity being described")
    predicate: str = Field(description="Predicate: The property or action of the subject that is being described")
    object: str = Field(description="Object: The value of the property or action being described")

class Facts(BaseModel):
    """A series of useful facts"""
    facts: List[Fact]


In [9]:
from langchain_openai import ChatOpenAI

llm_extractor=ChatOpenAI(temperature=0, model_name="gpt-4-turbo")

In [10]:
chain_extractor = extract_prompt | llm_extractor.with_structured_output(Facts)

In [11]:
facts = chain_extractor.invoke(pages[112].page_content)

In [12]:
def stringify_facts(f:Facts):
    res = set()
    for fact in f.facts:
        res.add(f'({fact.subject}) - [{fact.predicate}] -> ({fact.object})')
    return res

for s in stringify_facts(facts):
    print(s)

(Stability Control Indicator) - [action required] -> (check system)
(Transmission Tow/Haul Indicator) - [illuminates] -> (tow/haul activated)
(Service Engine Soon Indicator) - [illuminates] -> (engine malfunction)
(Service Engine Soon Indicator) - [action required] -> (service vehicle)
(Service Engine Soon Indicator) - [flashes] -> (not ready for I/M testing)
(Transmission Tow/Haul Indicator) - [flashes] -> (system service required)
(Stability Control Indicator) - [illuminates] -> (system malfunction)
(Stability Control Off Indicator) - [illuminates] -> (system off)
(Powertrain Malfunction Indicator) - [illuminates] -> (4WD fault detected)
(Powertrain Malfunction Indicator) - [illuminates] -> (powertrain fault detected)
(Service Engine Soon Indicator) - [action required] -> (drive moderately)
(Service Engine Soon Indicator) - [illuminates] -> (ignition on)
(Powertrain Malfunction Indicator) - [action required] -> (contact dealer)
(Service Engine Soon Indicator) - [illuminates] -> (read

In [13]:
relevance_prompt = ChatPromptTemplate.from_messages(
    [

        (
            "system",
            "You are an expert car mechanic that needs to validate the relevance and clarity of extracted facts from the "
            "Ford Super Duty Owners manual. "
            "The extracted facts will be loaded into a knowledge base for helping non-expert owners "
            "answer critical questions about their vehicle and provide them expert guidance. "
            "To be relevant, the fact should make sense, be non-ambiguous, and relevant to owners asking "
            "questions about their vehicle. "
            "In your response only return relevant facts according to this criteria."
            "Do not create fictitious data or impute missing values. "
            "Do not alter any of the facts. "
            "Only include facts from the ones provided."
        ),
        ("human", "{text}"),
    ]
)

In [14]:
chain_filterer = (relevance_prompt | llm_extractor.with_structured_output(Facts)).with_types(input_type=Facts, output_type=Facts)

In [15]:
filtered_facts = chain_filterer.invoke(facts)

In [16]:
print(f'Extracted Facts: {len(facts.facts)}')
print(f'Relevant Facts: {len(filtered_facts.facts)}')

Extracted Facts: 20
Relevant Facts: 20


In [17]:
facts_set = stringify_facts(facts)
filtered_facts_set = stringify_facts(filtered_facts)
print('Dropped facts:')
for fact in facts_set.difference(filtered_facts_set):
    print(fact)

Dropped facts:


## Create Classes and Functions for Pipeline

In [18]:
class Chunk(BaseModel):
    """A chunk of a source from which facts were extracted"""
    id:str
    sourceId:str
    seqId:int
    text:str


class Source(BaseModel):
    """A source of facts"""
    id:str
    name:str
    type:str
    url:str

class CitedFact(BaseModel):
    """A useful fact with a source chunk id"""
    subject: str
    predicate: str
    object: str
    chunkId: str
    def __init__(self, fact: Fact, chunk:Chunk):
        super().__init__(subject = fact.subject.lower(),
                         predicate = fact.predicate.lower(),
                         object = fact.object.lower(),
                         chunkId = chunk.id)


In [19]:
import uuid
from typing import Tuple
from langchain_core.documents import Document

def insert_sources(source:Source, chunks: List[Chunk]):
    #source node
    graph.query('''
    MERGE(s:Source {id:$id})
    SET s.name = $name, s.type=$type, s.url=$url
    ''', params=source.dict())

    #chunk nodes and rels to source
    graph.query('''
    UNWIND $chunks AS chunk
    MATCH(s:Source {id:chunk.sourceId})
    MERGE(c:Chunk {id:chunk.id})
    SET c.seqId=chunk.seqId, c.text=chunk.text
    MERGE (c)-[:PART_OF]->(s)
    ''', params={'chunks': [chunk.dict() for chunk in chunks]})

merge_entity_query = """
    UNWIND $entities AS entity
    MATCH(c:Chunk {id:entity.chunkId})
    MERGE(e:Entity {id:entity.id})
    MERGE (c)-[:HAS_ENTITY]->(e)
"""

def insert_facts(facts: List[CitedFact]):
    #insert subjects
    subjects = [{'id':fact.subject, 'chunkId':fact.chunkId} for fact in facts]
    graph.query(merge_entity_query, params={'entities': subjects})

    #insert objects
    objects = [{'id':fact.object, 'chunkId':fact.chunkId} for fact in facts]
    graph.query(merge_entity_query, params={'entities': objects})

    #insert predicates
    graph.query("""
    UNWIND $facts AS fact
    MATCH(s:Entity {id:fact.subject})
    MATCH(o:Entity {id:fact.object})
    MERGE (s)-[r:RELATES_TO {id:fact.predicate}]->(o)
    ON CREATE SET r.chunkId = fact.chunkId
    """, params={'facts': [fact.dict() for fact in facts]})


def create_sources(docs: List[Document], name: str, doc_type: str, url: str) -> Tuple[Source, List[Chunk]]:
    source_uuid = uuid.uuid3(uuid.NAMESPACE_URL, url)
    source = Source(id=str(source_uuid), name=name, type=doc_type, url=url)
    chunks = []
    for i in range(len(docs)):
        chunks.append(Chunk(
            id=str(uuid.uuid3(source_uuid, docs[i].page_content)),
            sourceId=source.id, seqId=i, text=docs[i].page_content))
    return source, chunks

def filter_facts(unfiltered_facts:Facts) -> Facts:
    facts = chain_filterer.invoke(unfiltered_facts)

    unfiltered_facts_len = len(unfiltered_facts.facts)
    facts_len = len(facts.facts)

    print(f'\tExtracted Facts: {unfiltered_facts_len}')
    print(f'\tRelevant Facts: {facts_len}')

    if unfiltered_facts_len != facts_len:
        unfiltered_facts_set = stringify_facts(unfiltered_facts)
        facts_set = stringify_facts(facts)
        print('\tDropped facts:')
        for dropped_fact in unfiltered_facts_set.difference(facts_set):
            print(f'\t\t * {dropped_fact}')
    return facts

def extract_facts(chunk: Chunk, filter:bool=True) -> List[CitedFact]:
    facts = chain_extractor.invoke(chunk.text)
    if filter:
        facts = filter_facts(facts)
    return [CitedFact(fact, chunk) for fact in facts.facts]

## Run Pipeline

In [20]:
from langchain_community.graphs.neo4j_graph import Neo4jGraph

graph = Neo4jGraph(NEO4J_URI, NEO4J_USERNAME, NEO4J_PASSWORD)
graph.query('CREATE CONSTRAINT entityId IF NOT EXISTS FOR (n:Entity) REQUIRE (n.id) IS UNIQUE;')
graph.query('CREATE CONSTRAINT chunkId IF NOT EXISTS FOR (n:Chunk) REQUIRE (n.id) IS UNIQUE;')
graph.query('CREATE CONSTRAINT sourceId IF NOT EXISTS FOR (n:Source) REQUIRE (n.id) IS UNIQUE;')


[]

In [21]:
###

In [22]:
%%time

from tqdm import tqdm

print("formatting and ingesting sources")
source, chunks = create_sources(pages[200:],
                                "Ford 2020 Super Duty Owner's Manual",
                                "pdf",
                                "https://www.fordservicecontent.com/Ford_Content/Catalog/owner_information/2020-Ford-F250-F350-F450-F550-F600-Owners-Manual-version-1._om_EN_10_2019.pdf")
insert_sources(source, chunks)

all_cited_facts = []
print("extracting facts")
for chunk in tqdm(chunks):
    try:
        cited_facts = extract_facts(chunk)
        insert_facts(cited_facts)
        all_cited_facts.extend(extract_facts(chunk))
    except Exception as e: print(e)
print("ingesting facts")
#insert_facts(all_cited_facts)
print("pipeline completed")

formatting and ingesting sources
extracting facts


  0%|          | 0/200 [00:00<?, ?it/s]

	Extracted Facts: 4
	Relevant Facts: 3
	Dropped facts:
		 * (LC3J 19A321 AA) - [is a model] -> (2020 SUPER DUTY Owner’s Manual)


  0%|          | 1/200 [00:19<1:04:27, 19.43s/it]

	Extracted Facts: 4
	Relevant Facts: 3
	Dropped facts:
		 * (LC3J 19A321 AA) - [is model number] -> (2020 SUPER DUTY Owner’s Manual)
	Extracted Facts: 17
	Relevant Facts: 13
	Dropped facts:
		 * (Publication) - [reproduction restriction] -> (written permission required)
		 * (Part Number) - [value] -> (201909 20190911200549)
		 * (Publication) - [correctness timing] -> (time of print)
		 * (Ford Motor Company) - [copyright year] -> (2019)


...

	Extracted Facts: 21
	Relevant Facts: 21
	Extracted Facts: 21
	Relevant Facts: 21


100%|█████████▉| 199/200 [1:59:07<00:40, 40.22s/it]

	Extracted Facts: 15
	Relevant Facts: 15
	Extracted Facts: 15
	Relevant Facts: 15


100%|██████████| 200/200 [1:59:39<00:00, 35.90s/it]

	Extracted Facts: 15
	Relevant Facts: 15
ingesting facts
pipeline completed
CPU times: user 15.9 s, sys: 1.59 s, total: 17.5 s
Wall time: 1h 59min 40s


In [23]:
import pickle

with open('manual2.pickle', 'ab') as f:
    pickle.dump(all_cited_facts, f)